# 编写自定义层和模型

In [2]:
import tensorflow as tf
from tensorflow import keras

## Layer类可自定义权重和一些计算
call方法对输入进行转换

In [10]:
class Linear(keras.layers.Layer):
    def __init__(self,units=32,input_dim=32):
        super().__init__()
        # add_weight(name,shape,dtype,initializer,regularizer,trainable) 常用参数
        self.w=self.add_weight(shape=(input_dim,units),dtype="float32",name="w",trainable=True,initializer="zeros")
        self.b=self.add_weight(shape=(units,),dtype="float32",name="b",trainable=True,initializer="zeros")
    def call(self,inputs):
        return tf.matmul(inputs,self.w)+self.b
    
x=tf.ones((2,2))
linear_layer=Linear(4,2)
y=linear_layer(x)
print(y)

tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(2, 4), dtype=float32)


In [6]:
y.shape

TensorShape([2, 4])

In [12]:
# 也可以定义非trainable的数据
class ComputeSum(keras.layers.Layer):
    def __init__(self,input_dim):
        super().__init__()
        self.total=self.add_weight(shape=(input_dim,),initializer="zeros",trainable=False,name="total")
    def call(self,inputs):
        self.total.assign_add(tf.reduce_sum(inputs,axis=0))
        return self.total
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())
print("weights:", my_sum.weights)
print("non-trainable weights:", my_sum.non_trainable_weights)

# It's not included in the trainable weights:
print("trainable_weights:", my_sum.trainable_weights)

[2. 2.]
[4. 4.]
weights: [<tf.Variable 'total:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]
non-trainable weights: [<tf.Variable 'total:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]
trainable_weights: []


** 上面的参数是在__init__里面指定的，但是实际应用中 我们会不专门指定大小，这就需要在传入输入后自动获取大小：在build方法里传入input的shape，然后在里面创建参数  **

`如下所示`

In [15]:
class Linear(keras.layers.Layer):
    def __init__(self,units=32):
        super().__init__()
        self.units=units
        
    def build(self,input_shape):
        """__call__方法会使得 在第一次调用时运行build方法，即创建对象不会运行 执行对象时会运行"""
        self.w=self.add_weight(shape=(input_shape[-1],self.units),dtype="float32",name="w",trainable=True,initializer="zeros")
        self.b=self.add_weight(shape=(self.units,),dtype="float32",name="b",trainable=True,initializer="zeros")
        
    def call(self,inputs):
        return tf.matmul(inputs,self.w)+self.b
    
x=tf.ones((2,2))
# 不运行build
linear_layer=Linear(4)
# 运行build
y=linear_layer(x)
print(y)

tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(2, 4), dtype=float32)


## 层内也是可以组合的
**  比如在一个Layer实现类中 增加多个Layer实现类 **

`示例如下`

In [16]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6


## add_loss 方法
** 当调用call方法，可以在其中添加add_loss 操作，这些添加的loss会追加到layer.losses里去，并且每次运行一次__call__ ，就会把losses中的loss重置**

In [17]:
class OuterLayer(keras.layers.Layer):
    def __init__(self,rate=1e-2):
        super().__init__()
        self.rate=rate
    def call(self,inputs):
        self.add_loss(self.rate*tf.reduce_sum(inputs))
        return inouts

## add_metric方法
** 和add_loss方法类似的使用，用来计算评估的内容 ** 

## get_config方法
** 可以定义这个方法返回layer的参数等信息，可以通过Layer.from_config的方法重构这个层 **

** 当你使用这个方法时，可以在init中调用父类的实现（传入dtype以及name），然后在get_config中调用父类的get_config得到config，再用config的update参数，将本层的参数更新进入config **

In [20]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config


layer = Linear(64)
print(layer)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)
print(new_layer)

{'name': 'linear_11', 'trainable': True, 'dtype': 'float32', 'units': 64}
